In [139]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

**drop unimportant column**

In [140]:
house_data = pd.read_csv('/content/drive/MyDrive/یادگیری ماشین/archive/Bengaluru_House_Data.csv')
#house_data.head(4)

In [141]:
house_data.drop(['area_type' , 'availability','society','balcony'] , axis=1 , inplace=True)

In [ ]:
house_data.isna().sum()

In [143]:
house_data.dropna(inplace=True)

In [144]:
#house_data.isna().sum()

In [145]:
#house_data.head(3)

In [ ]:
house_data.shape

In [ ]:
house_data['size'].unique()

In [ ]:
house_data.info()

**convert object columns  to numric ones type**

In [ ]:
house_data.bath.unique()

In [ ]:
# bath column
house_data.bath.astype('float32')

In [200]:
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True

In [ ]:
#show data that can not be convert to float easily
house_data[~ house_data['total_sqft'].apply(is_float)]

In [153]:
def edit_total_sqft(y):
  range_nums = y.split('-')
  if len(range_nums) ==2:
    mean_num = (float(range_nums[0]) + float(range_nums[1])) / 2
    return mean_num
  try:
      return float(y)
  except:
    return None

In [154]:
df_house = house_data.copy()
df_house['total_sqft'] = df_house['total_sqft'].apply(edit_total_sqft)

In [155]:
df_house.dropna(inplace=True)

In [156]:
def edit_size(z):
  num = z.split()
  return float(num[0])

In [157]:
bedroom = df_house['size'].apply(edit_size)
df_house['bedroom'] = bedroom

In [ ]:
df_house.head()
df_house.isna().sum()

In [159]:
df_house.head()

,location,size,total_sqft,bath,price,bedroom
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2.0
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4.0
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3.0
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3.0
4,Kothanur,2 BHK,1200.0,2.0,51.00,2.0


In [160]:
price_per_square = (df_house.price * 30000 / df_house.total_sqft)
df_house['price_sqft'] = price_per_square 

**handling unbalanced and  outliers** 

In [ ]:
df_house[df_house.bath == 40]

In [162]:
def remove_pps_outliers(df):
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('location'):
        mean_price = np.mean(subdf.price_sqft)
        strandard_deviation = np.std(subdf.price_sqft)
        reduced_df = subdf[(subdf.price_sqft>(mean_price-strandard_deviation)) & (subdf.price_sqft<=(mean_price+strandard_deviation))]
        df_out = pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out

In [163]:
df_house2 = remove_pps_outliers(df_house)

In [ ]:
df_house2.head()

In [166]:
def remove_bedroom_outliers(df):
    exclude_indices = np.array([])
    for location, location_df in df.groupby('location'):
        bhk_stats = {}
        for bhk, bhk_df in location_df.groupby('bedroom'):
            bhk_stats[bhk] = {
                'mean': np.mean(bhk_df.price_sqft),
                'std': np.std(bhk_df.price_sqft),
                'count': bhk_df.shape[0]
            }
        for bhk, bhk_df in location_df.groupby('bedroom'):
            stats = bhk_stats.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices = np.append(exclude_indices, bhk_df[bhk_df.price_sqft<(stats['mean'])].index.values)
    return df.drop(exclude_indices,axis='index')

In [167]:
df_house3 = remove_bedroom_outliers(df_house2)

In [ ]:
df_house3.head()

In [169]:
# handle location data
loc_stat = df_house3['location'].value_counts()
loc_less_than_10 = loc_stat[loc_stat <= 10]
df_house['location'] = df_house3['location'].apply(lambda z : 'other' if z in loc_less_than_10  else z)

numric_addres = pd.get_dummies(df_house3['location'])
numric_house_df = pd.concat([df_house3,numric_addres] , axis='columns')

In [ ]:
numric_house_df.head()

**split data**

In [171]:
X = numric_house_df.drop(['location','size','price'] , axis="columns")
Y = numric_house_df.price

**Hyperparameter tuning** 

In [ ]:
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import ShuffleSplit
def find_best_model_using_gridsearchcv(X,Y):
    algos = {
        'linear_regression' : {
            'model': LinearRegression(),
            'params': {
                'normalize': [True, False]
            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection': ['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion' : ['mse','friedman_mse'],
                'splitter': ['best','random']
            }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for algo_name, config in algos.items():
        gs =  GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
        gs.fit(X,Y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores,columns=['model','best_score','best_params'])
find_best_model_using_gridsearchcv(X,Y)

In [172]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(X,Y , test_size=0.15)

**bulid the model**

In [ ]:
from sklearn.tree import DecisionTreeRegressor
reg = DecisionTreeRegressor(criterion='mse' , splitter='best')
reg.fit(x_train , y_train)

In [ ]:
reg.score(x_test , y_test)

In [ ]:
y_hat = reg.predict(x_test)
y_hat[:5]

In [ ]:
y_test[:5]

**save model** 

In [ ]:
pip install pickle5


In [203]:
import pickle5

In [205]:
filename = '/content/drive/MyDrive/یادگیری ماشین/prediction_house_model.sav'
pickle5.dump(reg, open(filename, 'wb'))